### Optimal Tactical Pathing for UGV

In [69]:
import gurobipy as gp
from gurobipy import Model, GRB
import numpy as np
import scipy.sparse as sp
import pandas as pd
import os
import csv
from PIL import Image

In [70]:
def two_step_optimization(w, h, s, f, A, N, d, t, E, inflow, outflow, batteryCapacity):
    MAXTIME = 120  # Time limit for optimization
    MIPGAP = 1e-2  # Allowable MIP gap

    # Step 1: Minimize detection cost
    m1 = Model("Least_Detection_Path")
    m1.setParam("TimeLimit", MAXTIME)
    m1.setParam("MIPGap", MIPGAP)
    m1.setParam("OutputFlag", 1)

    # Decision variables: x[i] = 1 if arc i is used, 0 otherwise
    x = m1.addVars(A, vtype=GRB.BINARY, name="x")

    # Objective: Minimize detection cost
    m1.setObjective(sum(d[i] * x[i] for i in range(A)), GRB.MINIMIZE)

    # Flow conservation constraints
    print(N)

    # Ensure start and finish conditions
    m1.addConstr(sum(x[k] for k in inflow[s]) - sum(x[k] for k in outflow[s]) == -1)
    m1.addConstr(sum(x[k] for k in inflow[f]) - sum(x[k] for k in outflow[f]) == 1)

    for i in range(1,N+1):
        m1.addConstr(sum(x[k] for k in inflow[i]) <= 1)
        m1.addConstr(sum(x[k] for k in outflow[i]) <= 1)
        if i != f and i != s:
            m1.addConstr(sum(x[k] for k in inflow[i]) - sum(x[k] for k in outflow[i]) == 0)

    

    # Triangular constraint (no more than one triangle crossing)
    # for t_set in tris:
    #     m1.addConstr(sum(x[t] for t in t_set) <= 1)

    # Time constraint
    m1.addConstr(sum(t[i] * x[i] for i in range(A)) <= τ)

    # Solve model
    m1.optimize()

    # Extract solution
    optimal_path = {i: x[i].x for i in range(A)}
    path_cost = m1.objVal if m1.status == GRB.OPTIMAL else None

    print("Least detection path:", optimal_path)
    print("Detection cost:", path_cost)

def write_path(opt_path, A, scenario_name="default"):
    path = [i for i in range(A) if opt_path[i] == 1]
    path_df = pd.DataFrame(path, columns=["Path"])
    path_df.to_csv(f"output_{scenario_name}.csv", index=False)
    print(f"Path saved to output_{scenario_name}.csv")

In [71]:
SITUATION = "Buckner"
SPACING = 40

In [72]:
sat_path = os.path.join(os.getcwd(), 'imagery', SITUATION+'.png')
sat_map = np.asarray(Image.open(sat_path))
l, w, h = sat_map.shape
rows = l // SPACING
cols = w // SPACING
cols


3

In [73]:
arcs = pd.read_csv('Buckner_arcs_3_3.csv')
arcs
ins_csv = pd.read_csv('Buckner_ins_3_3.csv', header=None)
outs_csv = pd.read_csv('Buckner_outs_3_3.csv', header=None)

ins_csv


,0,1,2,3,4,5,6,7,8,9
0,9,18,26,53,0,0,0,0,0,0
1,1,13,19,27,33,59,0,0,0,0
2,5,28,34,63,0,0,0,0,0,0
3,3,8,25,37,42,69,0,0,0,0
4,2,7,12,15,32,38,43,47,78,0
5,6,11,21,44,48,84,0,0,0,0
6,17,24,41,88,0,0,0,0,0,0
7,16,23,31,36,46,94,0,0,0,0
8,22,30,40,98,0,0,0,0,0,0
9,4,58,67,75,0,0,0,0,0,0


In [74]:
inflow = ins_csv.apply(lambda row: row[row != 0].tolist(), axis=1).to_dict()
inflow = {
    idx + 1: row[row != 0].tolist()
    for idx, row in ins_csv.iterrows()
}


outflow = outs_csv.apply(lambda row: row[row != 0].tolist(), axis=1).to_dict()
outflow = {
    idx + 1: row[row != 0].tolist()
    for idx, row in outs_csv.iterrows()
}

In [75]:
inflow

{1: [9, 18, 26, 53],
 2: [1, 13, 19, 27, 33, 59],
 3: [5, 28, 34, 63],
 4: [3, 8, 25, 37, 42, 69],
 5: [2, 7, 12, 15, 32, 38, 43, 47, 78],
 6: [6, 11, 21, 44, 48, 84],
 7: [17, 24, 41, 88],
 8: [16, 23, 31, 36, 46, 94],
 9: [22, 30, 40, 98],
 10: [4, 58, 67, 75],
 11: [10, 50, 62, 68, 76, 82],
 12: [14, 54, 77, 83],
 13: [20, 52, 57, 74, 86, 91],
 14: [29, 51, 56, 61, 64, 81, 87, 92, 96],
 15: [35, 55, 60, 70, 93, 97],
 16: [39, 66, 73, 90],
 17: [45, 65, 72, 80, 85, 95],
 18: [49, 71, 79, 89]}

In [76]:
t = 60*60*60
s = 1
f = 9
N=cols*rows*2
A=max(arcs['Arc'])
d = arcs['Risk']
E = arcs['Energy Level']
t = 60*60*60

batteryCapacity = 100000
A

98

In [77]:
print("Inflow:", inflow)
print("Outflow:", outflow)

Inflow: {1: [9, 18, 26, 53], 2: [1, 13, 19, 27, 33, 59], 3: [5, 28, 34, 63], 4: [3, 8, 25, 37, 42, 69], 5: [2, 7, 12, 15, 32, 38, 43, 47, 78], 6: [6, 11, 21, 44, 48, 84], 7: [17, 24, 41, 88], 8: [16, 23, 31, 36, 46, 94], 9: [22, 30, 40, 98], 10: [4, 58, 67, 75], 11: [10, 50, 62, 68, 76, 82], 12: [14, 54, 77, 83], 13: [20, 52, 57, 74, 86, 91], 14: [29, 51, 56, 61, 64, 81, 87, 92, 96], 15: [35, 55, 60, 70, 93, 97], 16: [39, 66, 73, 90], 17: [45, 65, 72, 80, 85, 95], 18: [49, 71, 79, 89]}
Outflow: {1: [1, 2, 3, 4], 2: [5, 6, 7, 8, 9, 10], 3: [11, 12, 13, 14], 4: [15, 16, 17, 18, 19, 20], 5: [21, 22, 23, 24, 25, 26, 27, 28, 29], 6: [30, 31, 32, 33, 34, 35], 7: [36, 37, 38, 39], 8: [40, 41, 42, 43, 44, 45], 9: [46, 47, 48, 49], 10: [50, 51, 52, 53], 11: [54, 55, 56, 57, 58, 59], 12: [60, 61, 62, 63], 13: [64, 65, 66, 67, 68, 69], 14: [70, 71, 72, 73, 74, 75, 76, 77, 78], 15: [79, 80, 81, 82, 83, 84], 16: [85, 86, 87, 88], 17: [89, 90, 91, 92, 93, 94], 18: [95, 96, 97, 98]}


In [78]:
two_step_optimization(w, h, s, f, A, N, d, t, E, inflow, outflow, batteryCapacity)

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 0.01
Set parameter OutputFlag to value 1
18


KeyError: 98